# Download dos casos de Dengue

#### Os casos de Dengue foram baixados a partir do repositório do Sistema de Informação de Agravos de Notificação (SINAN),  que é alimentado, principalmente, pela notificação e investigação de casos de doenças e agravos que constam da lista nacional de doenças de notificação compulsória.  O downlod foi feito a partir do Package Python PySUS, criado para auxiliar no acesso e manipulação de base de dados publicadas pelo DATASUS.

#### Maiores informações em:

 - [SINAN] (https://portalsinan.saude.gov.br/sinan-net)
 - [Dengue] (https://portalsinan.saude.gov.br/dengue)
 - [PySUS] (https://pysus.readthedocs.io/en/latest/)


#### 1 - Importando o método SINAN do pacote PySUS

In [5]:
from pysus.ftp.databases.sinan import SINAN

#### 2 - Carregando os dados a partir do servidor do DataSUS

In [6]:
sinan = SINAN().load()

#### 3 - Lista os arquivos disponíveis para Dengue

In [7]:
arquivos = sinan.get_files(dis_code=["DENG"])

#### 4 - Download dos arquivos disponíveis

In [8]:
parquets = sinan.download(arquivos)

100%|██████████| 225M/225M [00:00<00:00, 26.3GB/s]


#### 5 - Converte os arquivos parquet em uma lista de dataframes

In [9]:
parquets

[/home/limafb/pysus/DENGBR00.parquet,
 /home/limafb/pysus/DENGBR01.parquet,
 /home/limafb/pysus/DENGBR02.parquet,
 /home/limafb/pysus/DENGBR03.parquet,
 /home/limafb/pysus/DENGBR04.parquet,
 /home/limafb/pysus/DENGBR05.parquet,
 /home/limafb/pysus/DENGBR06.parquet,
 /home/limafb/pysus/DENGBR07.parquet,
 /home/limafb/pysus/DENGBR08.parquet,
 /home/limafb/pysus/DENGBR09.parquet,
 /home/limafb/pysus/DENGBR10.parquet,
 /home/limafb/pysus/DENGBR11.parquet,
 /home/limafb/pysus/DENGBR12.parquet,
 /home/limafb/pysus/DENGBR13.parquet,
 /home/limafb/pysus/DENGBR14.parquet,
 /home/limafb/pysus/DENGBR15.parquet,
 /home/limafb/pysus/DENGBR16.parquet,
 /home/limafb/pysus/DENGBR17.parquet,
 /home/limafb/pysus/DENGBR18.parquet,
 /home/limafb/pysus/DENGBR19.parquet,
 /home/limafb/pysus/DENGBR20.parquet,
 /home/limafb/pysus/DENGBR21.parquet,
 /home/limafb/pysus/DENGBR22.parquet,
 /home/limafb/pysus/DENGBR23.parquet,
 /home/limafb/pysus/DENGBR24.parquet]

In [11]:
import os

In [13]:
# dataframes = [parquet.to_dataframe() for parquet in parquets]

for parquet in parquets:
    nome_parquet = os.path.splitext(os.path.basename(parquet))[0]
    print(nome_parquet)



    

TypeError: expected str, bytes or os.PathLike object, not ParquetSet

#### 6 - Concatena em um único dataframe selecionando apenas as colunas de interesse

In [ ]:
import pandas as pd

In [ ]:
selected_columns = ['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']
selected_dfs = [df[selected_columns] for df in dataframes]
df = pd.concat(selected_dfs, ignore_index=True, axis=0)

#### 7 - Corrige os tipos dos campos de data e numéricos, e decodifica e o campo da semana da notificação

In [ ]:
import numpy as np

In [ ]:
# Cria a função convert_week para decodificar a coluna "SEM_NOT" 
def convert_week(x):
        try:
            w = int(x) % 100
        except ValueError:
            w = np.nan
        return w


In [ ]:
#Converte os valores 'na' para '0'
df.fillna(0, inplace = True)

TypeError: Cannot set non-string value '0' into a StringArray.

In [ ]:
    for cname in df.columns:
        df[cname].replace("", np.nan, inplace=True)
        if cname.startswith(("NU", "ID", "SEM")):
            try:
                df[cname] = pd.to_numeric(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to numeric: {e}")
                # certain IDs can be alphanumerical
                pass
        elif cname.startswith("DT"):
            try:
                df[cname] = pd.to_datetime(df[cname])
            except ValueError as e:
                print(f"Column {cname} could not be converted to date: {e}")
        elif cname.startswith("SEM"):
            df[cname] = df[cname].map(convert_week)

/tmp/ipykernel_10861/1540400806.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[cname].replace("", np.nan, inplace=True)
/tmp/ipykernel_10861/1540400806.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [ ]:
df

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP
0,2007-01-30,<NA>,2007,292790
1,2007-01-14,200703,2007,292790
2,2007-01-18,<NA>,2007,292870
3,2007-01-04,200701,2007,291510
4,2007-01-24,200704,2007,291510
...,...,...,...,...
162605,2023-12-28,202352,2023,314200
162606,2023-12-29,202352,2023,351040
162607,2023-12-29,202352,2023,355030
162608,2023-12-29,202352,2023,261160


#### 8 - Agrupa e contando os registros por data e município

In [ ]:
counts = df.groupby(['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'ID_MUNICIP']).size().reset_index(name='count')

In [ ]:
counts

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP,count
0,2007-01-01,200701,2007,260400,1
1,2007-01-01,200701,2007,260410,2
2,2007-01-01,200701,2007,290800,1
3,2007-01-01,200701,2007,291220,1
4,2007-01-01,200701,2007,291390,4
...,...,...,...,...,...
93688,2023-12-28,202352,2023,316095,1
93689,2023-12-29,202352,2023,261160,1
93690,2023-12-29,202352,2023,314330,1
93691,2023-12-29,202352,2023,351040,1


#### 9 - Ligação com a tabela de municípios importada com o pacote Geobr
#### Até o ano de 2006, o código do município apresenta 7 dígitos, depois de 2006 o código do município apresenta 6 dígitos. Desta maneira o merge será dividido em duas etapas

In [ ]:
from geobr import read_municipality

In [ ]:
mun = read_municipality(code_muni="all", year=2022)

In [ ]:
# Converte a coluna 'code_muni' de float para int em uma nova coluna 'ID_MUNICIP'
mun['ID_MUNICIP_1'] = mun['code_muni'].astype('Int64')

# Remover o último dígito da coluna 'ID_MUNICIP'
mun['ID_MUNICIP_2'] = mun['ID_MUNICIP_1'].astype(str).str[:-1].astype(int)

In [ ]:
mun

,code_muni,name_muni,code_state,abbrev_state,name_state,code_region,name_region,geometry,ID_MUNICIP_1,ID_MUNICIP_2
0,1100015.0,Alta Floresta D'oeste,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-61.96836 -12.13407, -61.96827...",1100015,110001
1,1100023.0,Ariquemes,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-63.18281 -10.13892, -63.18615...",1100023,110002
2,1100031.0,Cabixi,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-60.70518 -13.32474, -60.70904...",1100031,110003
3,1100049.0,Cacoal,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-61.34740 -11.50562, -61.34569...",1100049,110004
4,1100056.0,Cerejeiras,11.0,RO,Rondônia,1.0,Norte,"MULTIPOLYGON (((-60.82417 -13.11156, -60.82289...",1100056,110005
...,...,...,...,...,...,...,...,...,...,...
5567,5222005.0,Vianópolis,52.0,GO,Goiás,5.0,Centro Oeste,"MULTIPOLYGON (((-48.30928 -16.84149, -48.30293...",5222005,522200
5568,5222054.0,Vicentinópolis,52.0,GO,Goiás,5.0,Centro Oeste,"MULTIPOLYGON (((-49.73218 -17.74154, -49.73386...",5222054,522205
5569,5222203.0,Vila Boa,52.0,GO,Goiás,5.0,Centro Oeste,"MULTIPOLYGON (((-47.07533 -15.06258, -47.07448...",5222203,522220
5570,5222302.0,Vila Propício,52.0,GO,Goiás,5.0,Centro Oeste,"MULTIPOLYGON (((-48.58701 -15.31582, -48.58952...",5222302,522230


In [ ]:
# Junção dos dataframes 'counts' e 'mun' mantendo todos os registros mesmo quando não houver correspondência
merged = counts.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_2', how='outer', indicator=True)
#merged_2 = counts_2.merge(mun, left_on='ID_MUNICIP', right_on='ID_MUNICIP_2', how='outer', indicator=True)

In [ ]:
merged.describe()

,DT_NOTIFIC,SEM_NOT,NU_ANO,ID_MUNICIP,count,code_muni,code_state,code_region,ID_MUNICIP_1,ID_MUNICIP_2
count,93693,93693.0,93693.0,93693.0,93693.000000,9.694800e+04,96948.000000,96948.000000,96948.0,96948.000000
mean,2012-08-28 08:20:46.761230848,199797.403317,2012.161314,309124.484401,1.704087,3.100450e+06,30.762295,2.722078,3100449.885629,310044.534947
min,2007-01-01 00:00:00,910.0,2007.0,110001.0,1.000000,1.100015e+06,11.000000,1.000000,1100015.0,110001.000000
25%,2009-03-02 00:00:00,200903.0,2009.0,292290.0,1.000000,2.922102e+06,29.000000,2.000000,2922102.0,292210.000000
50%,2011-04-14 00:00:00,201113.0,2011.0,313130.0,1.000000,3.131752e+06,31.000000,3.000000,3131752.5,313175.000000
75%,2015-08-11 00:00:00,201531.0,2015.0,317090.0,2.000000,3.200201e+06,32.000000,3.000000,3200201.0,320020.000000
max,2023-12-29 00:00:00,202352.0,2023.0,530010.0,436.000000,5.300108e+06,53.000000,5.000000,5300108.0,530010.000000
std,NaN,16945.43849,4.474205,38938.819399,2.861346,4.373192e+05,4.336067,0.587042,437319.192261,43731.910197


#### 10 - Exportando em CSV

In [ ]:
# Reordenando as colunas
sinan_esquisto_2007_2023 = merged[['DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'code_muni', 'name_muni', 'abbrev_state', 'name_state', 'name_region', 'count', 'geometry']]

# Renomeando as colunas
sinan_esquisto_2007_2023.columns = ['data', 'semana', 'ano','geocodigo', 'municipio',  'uf_sigla', 'uf', 'regiao', 'casos', 'geometry']


In [ ]:
# Exportando para CSV
sinan_esquisto_2007_2023.to_csv('sinan_esquisto_2007_2023.csv',sep=';')

#### 11 - Exportando em GeoJason

In [ ]:
# Importando geopandas
import geopandas as gpd

In [ ]:
# Convertendo para GeoDataFrame
gdf = gpd.GeoDataFrame(sinan_esquisto_2007_2023)

In [ ]:
# Exportanod para GeoJSON
gdf.to_file('sinan_esquisto_2007_2023.geojson', driver='GeoJSON')